In [1]:
import pandas as pd
import numpy as np
import warnings
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Data Cleaning

In [2]:
#reading the csv file into a dataframe 
shootings_df = pd.read_csv('usmassdata.csv', encoding='latin-1')
shootings_df.head()

,Case,Location,Date,Year,Summary,Fatalities,Injured,Total victims,Venue,Prior signs of mental health issues,...,Where obtained,Type of weapons,Weapon details,Race,Gender,Sources,Mental Health Sources,latitude,longitude,Type
0,Yountville veterans home shooting,"Yountville, CA",3/9/18,2018,"Army veteran Albert Cheung Wong, 36, stormed a...",3,0,3,Workplace,Yes,...,TBD,semiautomatic rifle; shotgun,NaN,Asian,M,https://www.cnn.com/2018/03/10/us/california-v...,https://www.cnn.com/2018/03/10/us/california-v...,NaN,NaN,Mass
1,Stoneman Douglas High School shooting,"Parkland, Florida",2/14/18,2018,"Nikolas J. Cruz, 19, heavily armed with an AR-...",17,14,31,School,Yes,...,A Florida pawn shop,semiautomatic rifle,AR-15,White,M,https://www.nytimes.com/2018/02/14/us/parkland...,https://www.nytimes.com/2018/02/15/us/nikolas-...,NaN,NaN,Mass
2,Pennsylvania carwash shooting,"Melcroft, PA",1/28/18,2018,"Timothy O'Brien Smith, 28, wearing body armor ...",4,1,5,Other,TBD,...,TBD,semiautomatic rifle and semiautomatic handgun,NaN,White,M,http://www.wpxi.com/news/top-stories/family-me...,NaN,NaN,NaN,Mass
3,Rancho Tehama shooting spree,"Rancho Tehama, CA",11/14/17,2017,"Kevin Janson Neal, 44, went on an approximatel...",5,10,15,Other,TBD,...,TBD,semiautomatic rifles,Two illegally modified rifles,White,M,https://www.nbcnews.com/news/us-news/californi...,NaN,NaN,NaN,Spree
4,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46+,Religious,Yes,...,Purchased in April 2016 from an Academy Sports...,semiautomatic rifle,Ruger AR-556; Kelley also possessed semiautoma...,White,M,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,32.780105,-96.800008,Mass


In [7]:
#testing for Geopy to retrieve (latitude , longitude) based on the "Location" field of the dataframe 
test_address = "Las Vegas, NV"
geolocator = Nominatim()
try:
    location = geolocator.geocode(test_address)
    print((location.latitude, location.longitude))
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(test_address, e.message))

(36.1662859, -115.149225)


In [4]:
# Loop through the dataframe to check if the latitude is missing. 
# If the Latitue is a "NaN" then update the (Lat, Lon) data from Geopy
for index, row in shootings_df.iterrows():
        try:
            location = geolocator.geocode(row['Location'])
            shootings_df.loc[index, 'latitude']= location.latitude
            shootings_df.loc[index, 'longitude']= location.longitude
        except GeocoderTimedOut as e:
            print("Error: geocode failed on input %s with message %s"%(row['Location'], e.message))      

In [6]:
# adding redults to another dataframe so that there is no confusion when referencing for Database creation 
complete_shootings_df = shootings_df
complete_shootings_df.head(20)      

,Case,Location,Date,Year,Summary,Fatalities,Injured,Total victims,Venue,Prior signs of mental health issues,...,Where obtained,Type of weapons,Weapon details,Race,Gender,Sources,Mental Health Sources,latitude,longitude,Type
0,Yountville veterans home shooting,"Yountville, CA",3/9/18,2018,"Army veteran Albert Cheung Wong, 36, stormed a...",3,0,3,Workplace,Yes,...,TBD,semiautomatic rifle; shotgun,NaN,Asian,M,https://www.cnn.com/2018/03/10/us/california-v...,https://www.cnn.com/2018/03/10/us/california-v...,38.395445,-122.360173,Mass
1,Stoneman Douglas High School shooting,"Parkland, Florida",2/14/18,2018,"Nikolas J. Cruz, 19, heavily armed with an AR-...",17,14,31,School,Yes,...,A Florida pawn shop,semiautomatic rifle,AR-15,White,M,https://www.nytimes.com/2018/02/14/us/parkland...,https://www.nytimes.com/2018/02/15/us/nikolas-...,26.310079,-80.237270,Mass
2,Pennsylvania carwash shooting,"Melcroft, PA",1/28/18,2018,"Timothy O'Brien Smith, 28, wearing body armor ...",4,1,5,Other,TBD,...,TBD,semiautomatic rifle and semiautomatic handgun,NaN,White,M,http://www.wpxi.com/news/top-stories/family-me...,NaN,40.052018,-79.389202,Mass
3,Rancho Tehama shooting spree,"Rancho Tehama, CA",11/14/17,2017,"Kevin Janson Neal, 44, went on an approximatel...",5,10,15,Other,TBD,...,TBD,semiautomatic rifles,Two illegally modified rifles,White,M,https://www.nbcnews.com/news/us-news/californi...,NaN,39.993786,-122.471880,Spree
4,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46+,Religious,Yes,...,Purchased in April 2016 from an Academy Sports...,semiautomatic rifle,Ruger AR-556; Kelley also possessed semiautoma...,White,M,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,29.274044,-98.055906,Mass
5,Walmart shooting in suburban Denver,"Thornton, CO",11/1/17,2017,"Scott Allen Ostrem, 47, walked into a Walmart ...",3,0,3,Other,Unclear,...,NaN,semiautomatic handgun,NaN,White,M,https://www.nytimes.com/2017/11/01/us/thornton...,NaN,39.917473,-104.905461,Mass
6,Edgewood businees park shooting,"Edgewood, MD",10/18/17,2017,"Radee Labeeb Prince, 37, fatally shot three pe...",3,3,6,Workplace,Unclear,...,Unclear,handgun,.380-caliber; make unclear,Black,M,http://www.baltimoresun.com/news/maryland/harf...,NaN,39.418719,-76.294402,Mass
7,Las Vegas Strip massacre,"Las Vegas, NV",10/1/17,2017,"Stephen Craig Paddock, 64, rained a barrage of...",58,700+,758+,Other,TBD,...,Two gun shops in Nevada,"23 firearms, mostly rifles; including scopes, ...","AR-15-style and AK-47-style rifles and ""a larg...",White,M,https://www.lvmpd.com/en-us/Documents/1_Octobe...,https://www.nytimes.com/2017/10/13/us/stephen-...,36.166286,-115.149225,Mass
8,San Francisco UPS shooting,"San Francisco, CA",6/14/17,2017,"Jimmy Lam, 38, fatally shot three coworkers an...",3,2,5,Workplace,Yes,...,Unclear; the firearm was stolen in Utah. A sec...,two handguns,MAC-10-style Òassault pistolÓ; 30-round magazi...,Asian,M,http://www.nbcbayarea.com/news/local/Active-Sh...,http://www.ktvu.com/news/ktvu-local-news/26177...,46.844325,-71.274327,Mass
9,Pennsylvania supermarket shooting,"Tunkhannock, PA",6/7/17,2017,"Randy Stair, a 24-year-old worker at Weis groc...",3,0,3,Workplace,Unclear,...,NaN,shotguns,NaN,White,M,http://www.pressconnects.com/story/news/local/...,NaN,41.538516,-75.946844,Mass


In [6]:
# list of all column headers
list(complete_shootings_df)

['Case',
 'Location',
 'Date',
 'Year',
 'Summary',
 'Fatalities',
 'Injured',
 'Total victims',
 'Venue',
 'Prior signs of mental health issues',
 'Mental health - details',
 'Weapons obtained legally',
 'Where obtained',
 'Type of weapons',
 'Weapon details',
 'Race',
 'Gender',
 'Sources',
 'Mental Health Sources',
 'latitude',
 'longitude',
 'Type']

In [8]:
#rename column headers so that there are no spaces

complete_shootings_df = complete_shootings_df.rename(columns={'Case': 'case', 
                                        'Location': 'location', 
                                        'Date': 'date', 
                                        'Year': 'year', 
                                        'Summary': 'summary', 
                                        'Fatalities': 'fatalities', 
                                        'Injured': 'injured', 
                                        'Total victims': 'total_victims', 
                                        'Venue': 'venue', 
                                        'Prior signs of mental health issues': 'prior_signs_mental_health', 
                                        'Mental health - details': 'mental_health', 
                                        'Weapons obtained legally': 'weapons_obtained_legally', 
                                        'Where obtained': 'where_obtained',
                                        'Type of weapons': 'type_weapon', 
                                        'Weapon details': 'weapon_details', 
                                        'Race': 'race', 
                                        'Gender': 'gender', 
                                        'Sources': 'sources', 
                                        'Mental Health Sources': 'mental_health_sources', 
                                        'Type': 'shooting_type'})

complete_shootings_df.head()

,case,location,date,year,summary,fatalities,injured,total_victims,venue,prior_signs_mental_health,...,where_obtained,type_weapon,weapon_details,race,gender,sources,mental_health_sources,latitude,longitude,shooting_type
0,Yountville veterans home shooting,"Yountville, CA",3/9/18,2018,"Army veteran Albert Cheung Wong, 36, stormed a...",3,0,3,Workplace,Yes,...,TBD,semiautomatic rifle; shotgun,NaN,Asian,M,https://www.cnn.com/2018/03/10/us/california-v...,https://www.cnn.com/2018/03/10/us/california-v...,38.395445,-122.360173,Mass
1,Stoneman Douglas High School shooting,"Parkland, Florida",2/14/18,2018,"Nikolas J. Cruz, 19, heavily armed with an AR-...",17,14,31,School,Yes,...,A Florida pawn shop,semiautomatic rifle,AR-15,White,M,https://www.nytimes.com/2018/02/14/us/parkland...,https://www.nytimes.com/2018/02/15/us/nikolas-...,26.310079,-80.237270,Mass
2,Pennsylvania carwash shooting,"Melcroft, PA",1/28/18,2018,"Timothy O'Brien Smith, 28, wearing body armor ...",4,1,5,Other,TBD,...,TBD,semiautomatic rifle and semiautomatic handgun,NaN,White,M,http://www.wpxi.com/news/top-stories/family-me...,NaN,40.052018,-79.389202,Mass
3,Rancho Tehama shooting spree,"Rancho Tehama, CA",11/14/17,2017,"Kevin Janson Neal, 44, went on an approximatel...",5,10,15,Other,TBD,...,TBD,semiautomatic rifles,Two illegally modified rifles,White,M,https://www.nbcnews.com/news/us-news/californi...,NaN,39.993786,-122.471880,Spree
4,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46+,Religious,Yes,...,Purchased in April 2016 from an Academy Sports...,semiautomatic rifle,Ruger AR-556; Kelley also possessed semiautoma...,White,M,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,29.274044,-98.055906,Mass


In [9]:
# recheck column header

complete_shootings_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 22 columns):
case                         98 non-null object
location                     98 non-null object
date                         98 non-null object
year                         98 non-null int64
summary                      98 non-null object
fatalities                   98 non-null int64
injured                      98 non-null object
total_victims                98 non-null object
venue                        98 non-null object
prior_signs_mental_health    98 non-null object
mental_health                87 non-null object
weapons_obtained_legally     98 non-null object
where_obtained               86 non-null object
type_weapon                  98 non-null object
weapon_details               91 non-null object
race                         96 non-null object
gender                       98 non-null object
sources                      98 non-null object
mental_health_sources        76 non

# Database Creation

In [9]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [10]:
# Create an engine to a SQLite database file called `shootings.sqlite`
engine = create_engine("sqlite:///shootings.sqlite")

In [12]:
# Create a session
session = Session(engine)

In [13]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [15]:
#append data from csv created df to correct classes(tables)
complete_shootings_df.to_sql('shootings', engine, if_exists='append', index=False)

In [16]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()

['shootings']

In [17]:
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('shootings')
for column in columns:
    print(column["name"], column["type"])

case TEXT
location TEXT
date TEXT
year BIGINT
summary TEXT
fatalities BIGINT
injured TEXT
total_victims TEXT
venue TEXT
prior_signs_mental_health TEXT
mental_health TEXT
weapons_obtained_legally TEXT
where_obtained TEXT
type_weapon TEXT
weapon_details TEXT
race TEXT
gender TEXT
sources TEXT
mental_health_sources TEXT
latitude FLOAT
longitude FLOAT
shooting_type TEXT
